## Module 3 Homework: Data Warehousing & BigQuery

In [37]:
# Import necessay libraries
import os
from google.cloud import bigquery
from google.cloud import storage
import pyarrow.parquet as pq
import gcsfs

For this homework we will be using the `Yellow Taxi Trip Records for January 2024 - June 2024` (not the entire year of data).

Parquet Files are available from the New York City Taxi Data found here:

https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page

### Question 1. Counting records
What is count of records for the 2024 Yellow Taxi Data?

In [ ]:
total = 0
for month in ["01","02","03","04","05","06"]:
    path = f"gs://kachi_dezoomcamp_hw3_2026/yellow_tripdata_2024-{month}.parquet"
    table = pq.read_table(path, filesystem=fs)
    total += table.num_rows

print("Total records (Jan-Jun 2024):", total)

Total records (Jan-Jun 2024): 20332093


### Question 2. Data read estimation

Write a query to count the distinct number of PULocationIDs for the entire dataset on both the tables.

`SELECT COUNT(DISTINCT PULocationID)`

``FROM `zoomcamp-ny-taxi.zoomcamp_hw3.yellow_taxi_2024`;``

What is the estimated amount of data that will be read when this query is executed on the External Table and the Table?

- 0 MB for the External Table 
- 155.12 MB for the Materialized Table

#### Question 4. 
How many records have a fare_amount of 0?

In [ ]:
client = bigquery.Client(project="zoomcamp-ny-taxi-486822")

query = """
SELECT COUNT(*) AS zero_fare_records
FROM `zoomcamp-ny-taxi-486822.zoomcamp_hw3.yellow_taxi_2024_bq`
WHERE fare_amount = 0
"""

job = client.query(query)
result = job.result()

for row in result:
    print("Zero fare records:", row.zero_fare_records)


Zero fare records: 8333


#### Question 5

What is the best strategy to make an optimized table in Big Query if your query will always filter based on tpep_dropoff_datetime and order the results by VendorID? 

Create a new table with this strategy:

CREATE OR REPLACE TABLE `zoomcamp-ny-taxi-486822.zoomcamp_hw3.yellow_taxi_2024_optimized`

PARTITION BY DATE(tpep_dropoff_datetime)

CLUSTER BY VendorID AS

SELECT *
FROM `zoomcamp-ny-taxi-486822.zoomcamp_hw3.yellow_taxi_2024_bq`;


---
#### Question 6

Write a query to retrieve the distinct VendorIDs between tpep_dropoff_datetime 2024-03-01 and 2024-03-15 (inclusive). Use the materialized table you created earlier in your from clause and note the estimated bytes. Now change the table in the from clause to the partitioned table you created for question 5 and note the estimated bytes processed. What are these values? 

a. **Materialized table : 310.24 MB**

b. **Partitioned table : 26.84 MB**

---
#### Question 7.

Where is the data stored in the External Table you created? 

**It is stored in Google Cloud Storage (GCS), specifically in the google bucket.**

---
#### Question 8. 

It is best practice in Big Query to always cluster your data:

**False**